# titanic生存預測
#### 參考資料 [[資料分析&機器學習] 第4.1講 : Kaggle競賽-鐵達尼號生存預測(前16%排名)](https://medium.com/@yehjames/%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90-%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92-%E7%AC%AC4-1%E8%AC%9B-kaggle%E7%AB%B6%E8%B3%BD-%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E7%94%9F%E5%AD%98%E9%A0%90%E6%B8%AC-%E5%89%8D16-%E6%8E%92%E5%90%8D-a8842fea7077)
#### [CH.Tseng Kaggle – Titanic倖存預測 #1](https://chtseng.wordpress.com/2017/12/24/kaggle-titanic%E5%80%96%E5%AD%98%E9%A0%90%E6%B8%AC-1/)、 [CH.Tseng Kaggle – Titanic倖存預測 #2](https://chtseng.wordpress.com/2017/12/25/kaggle-titanic%E5%80%96%E5%AD%98%E9%A0%90%E6%B8%AC-2/)

## 載模組&資料

In [136]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import cross_val_predict
from sklearn import metrics, cross_validation

import pandas as pd
import numpy as np
%matplotlib inline

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv('gender_submission.csv')

## 觀察資料

In [9]:
train.head() 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
train.info() # Age、Cabin、Embarked有缺失值

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [13]:
test.info() # Age、Fare、Cabin有缺失值

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [14]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [15]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


## 選特徵&補缺值

In [121]:
Y_train = train['Survived']
X_train = train.append(test) #合併表格處理，之後會再分開
X_train.shape

(1309, 12)

In [122]:
X_train['Family'] = X_train['SibSp'] + X_train['Parch'] # 把親屬合併起來，成為一個新特徵

X_train.drop('SibSp', 1, inplace=True)
X_train.drop('Parch', 1, inplace=True)

In [123]:
# 選特徵(我覺得簡單重要的&網站已分析的)
selected_features = ['Pclass', 'Age', 'Family', 'Fare', 'Sex', 'Embarked']

# 補缺值(簡單做或參考網站作法)，之後應仔細分析資料再個別處理
X_train['Age'].fillna(X_train['Age'].mean(), inplace=True)
X_train['Fare'].fillna(X_train['Fare'].median(), inplace=True)
X_train['Embarked'].fillna("C", inplace=True)


In [124]:
# One-Hot Encoding
X_train['Pclass'] = X_train['Pclass'].astype('category').cat.codes
X_train['Sex'] = X_train['Sex'].astype('category').cat.codes
X_train['Embarked'] = X_train['Embarked'].astype('category').cat.codes

In [125]:
# 準備訓練
X_test = X_train.iloc[891:]
X_train = X_train.head(891)

X_train.shape, Y_train.shape, X_test.shape


((891, 11), (891,), (418, 11))

In [126]:
X_train = X_train[selected_features]
X_train.head()

,Pclass,Age,Family,Fare,Sex,Embarked
0,2,22.0,1,7.2500,1,2
1,0,38.0,1,71.2833,0,0
2,2,26.0,0,7.9250,0,2
3,0,35.0,1,53.1000,0,2
4,2,35.0,0,8.0500,1,2


In [150]:
logreg = LogisticRegression()
Y_pred = cross_val_score(logreg, X_train, Y_train, cv=10).mean()
Y_pred

0.79802264215185559

In [151]:
# 輸出CSV
submission=pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived':Y_pred})

submission.to_csv('submission.csv', index=False)